#### **PROVA HUGGINFACE**


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import torch

In [34]:
import torch

if torch.cuda.is_available():
    print("GPU attiva:", torch.cuda.get_device_name(0))
else:
    print("Nessuna GPU trovata; si usa la CPU.")

Nessuna GPU trovata; si usa la CPU.


In [21]:
df_train = pd.read_csv("amazon_review_full_csv/train.csv", delimiter=';', encoding='latin1')  
# HOW MANY ROWS AND COLUMNS?
print("Dataset shape:", df_train.shape)
# LET'S SEE SOME ROWS
df_train.head(10)

Dataset shape: (1048575, 3)


,Rating,Title,Review
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...
5,5,There's a reason for the price,"There's a reason this CD is so expensive, even..."
6,1,Buyer beware,"This is a self-published book, and if you want..."
7,4,"Errors, but great story",I was a dissapointed to see errors on the back...
8,1,The Worst!,A complete waste of time. Typographical errors...
9,1,Oh please,I guess you have to be a romance novel lover f...


In [5]:
df_test = pd.read_csv("amazon_review_full_csv/test.csv", delimiter=';', encoding='latin1')  
# HOW MANY ROWS AND COLUMNS?
print("Dataset shape:", df_test.shape)
# LET'S SEE SOME ROWS
df_test.head(10)

Dataset shape: (650000, 3)


,Rating,Title,Review
0,1,mens ultrasheer,"This model may be ok for sedentary types, but ..."
1,4,Surprisingly delightful,This is a fast read filled with unexpected hum...
2,2,"Works, but not as advertised",I bought one of these chargers..the instructio...
3,2,Oh dear,I was excited to find a book ostensibly about ...
4,2,Incorrect disc!,"I am a big JVC fan, but I do not like this mod..."
5,2,Incorrect Disc,"I love the style of this, but after a couple y..."
6,2,DVD menu select problems,I cannot scroll through a DVD menu that is set...
7,3,My 2 y/o grandson loves it!!,This movie with all of its animals really keep...
8,5,A Cookbook Every Baker Should Own,I found a copy of this cookbook at a local use...
9,3,good basic,"The book is a basic ""how to"" book for using so..."


In [6]:
from transformers import pipeline 

# Load sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Example usage
example_review = "This is a fast read filled with unexpected humour and profound insights into the art of politics and policy. In brief, it is sly, wry, and wise."
result = sentiment_pipeline(example_review)

print(result)

Device set to use cpu


[{'label': '4 stars', 'score': 0.5392228960990906}]


In [23]:

# Unisci titolo + contenuto
df_train["Text"] = df_train["Title"].fillna("") + " " + df_train["Review"].fillna("")
df_train = df_train[["Text", "Rating"]]
df_train["Rating"] = df_train["Rating"] - 1
df_train.head(10)

,Text,Rating
0,more like funchuck Gave this to my dad for a g...,2
1,Inspiring I hope a lot of people hear this cd....,4
2,The best soundtrack ever to anything. I'm read...,4
3,Chrono Cross OST The music of Yasunori Misuda ...,3
4,Too good to be true Probably the greatest soun...,4
5,There's a reason for the price There's a reaso...,4
6,"Buyer beware This is a self-published book, an...",0
7,"Errors, but great story I was a dissapointed t...",3
8,The Worst! A complete waste of time. Typograph...,0
9,Oh please I guess you have to be a romance nov...,0


#### Here I split the training in Training and Validation

In [25]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_train["Text"].tolist(),
    df_train["Rating"].tolist(),
    test_size=0.2,
    stratify=df_train["Rating"],
    random_state=42
)

#### Tokenize

In [27]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

#### Create dataset with PyTorch

In [28]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }
    def __len__(self):
        return len(self.labels)

train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)

#### Upload the model for 5 classes

In [29]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "nlptown/bert-base-multilingual-uncased-sentiment",
    num_labels=5  # Per 5 classi (1-5 stelle)
)

In [30]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
import numpy as np

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    load_best_model_at_end=True,
    logging_steps=10,                    # Logga ogni 10 step
    report_to="none",                    # Evita problemi con wandb o tensorboard
    disable_tqdm=False                   # << ABILITA LA BARRA DI CARICAMENTO
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)



C:\Users\molna\anaconda3\envs\env_molny0\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [32]:
trainer.evaluate()

KeyboardInterrupt: 